In [2]:
# Import components
import datetime as dt
import math
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pmdarima as pm
import seaborn as sns
from matplotlib import pyplot as pltw
from pmdarima import arima, datasets, model_selection, utils
from scipy.spatial import distance
from sklearn import metrics
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import LocalOutlierFactor
from statsmodels.tsa.seasonal import STL

warnings.simplefilter("ignore")

plt.rcParams["font.family"] = "Hiragino Maru Gothic Pro"
plt.style.use("ggplot")  # グラフのスタイル
plt.rcParams["figure.figsize"] = [12, 9]  # グラフサイズ設定

In [1]:
def DDTW(Q, C):
    """
    Args:
        Q (np.array or list): 一つ目の波形
        C (np.array or list): 二つ目の波形

    Returns:
        γ_mat (np.array): DDTWを計算するための行列
        arrows (np.array): 各時点で←・↙︎・↓のどのマスが最小だったかを示す記号を保存する行列
        ddtw (float): DDTW
    """
    Q, C = np.array(Q), np.array(C)
    assert Q.shape[0] > 3, "一つ目の波形のフォーマットがおかしいです。"
    assert C.shape[0] > 3, "二つ目の波形のフォーマットがおかしいです。"

    # 3.1 Algorithm details の式
    def _Dq(q):
        return ((q[1] - q[0]) + (q[2] - q[0]) / 2) / 2

    # 二つの時点間の距離
    def _γ(x, y):
        return abs(_Dq(x) - _Dq(y))

    # 各変数
    n, m = Q.shape[0] - 2, C.shape[0] - 2
    γ_mat = np.zeros((n, m))
    arrows = np.array(np.zeros((n, m)), dtype=str)  # 可視化用の行列でDDTWの値とは無関係

    # 一番左下のスタート地点
    γ_mat[0, 0] = _γ(Q[0:3], C[0:3])

    # 一列目を計算
    for i in range(1, n):
        γ_mat[i, 0] = γ_mat[i - 1, 0] + _γ(Q[i - 1 : i + 2], C[0:3])
        arrows[i, 0] = "↓"

    # 一行目を計算
    for j in range(1, m):
        γ_mat[0, j] = γ_mat[0, j - 1] + _γ(Q[0:3], C[j - 1 : j + 2])
        arrows[0, j] = "←"

    # 残りのマスを計算
    for i in range(1, n):
        for j in range(1, m):
            # DDTWを求めるためのマトリクスを埋める
            d_ij = _γ(Q[i - 1 : i + 2], C[j - 1 : j + 2])
            γ_mat[i, j] = d_ij + np.min(
                [γ_mat[i - 1, j - 1], γ_mat[i - 1, j], γ_mat[i, j - 1]]
            )

            # 矢印を書くための行列(DDTWの値とは関係無い処理)
            if (
                square_index := np.argmin(
                    [γ_mat[i - 1, j - 1], γ_mat[i - 1, j], γ_mat[i, j - 1]]
                )
            ) == 0:
                arrows[i, j] = "↙︎"
            elif square_index == 1:
                arrows[i, j] = "↓"
            elif square_index == 2:
                arrows[i, j] = "←"

    return γ_mat, arrows, γ_mat[n - 1, m - 1]

In [3]:
df = pd.read_csv("../datasets/fixed_battery_log_2.csv")
df["date"] = pd.to_datetime(df["date"])
users = df["User"].unique()

In [20]:
bins = np.linspace(0, 100, 11)

In [10]:
first_date_cand = pd.date_range("2022-01-01", "2022-02-01")
score_G = []
score_I = []

days = [7,14, 21, 30]
weekdays = df["weekday"].unique()
data = df
clf = LocalOutlierFactor(n_neighbors=2)
bins = np.linspace(0, 100, 11)

for a in range(len(days)):
    diff_day_1 = days[a]
    for b in range(len(days)):
        diff_day_2 = days[b]
        for h in range(len(users)):
            user = users[h]
            for i in range(0, 101):
                first_date = first_date_cand[
                    np.random.randint(0, len(first_date_cand), 1)
                ]
                last_date = first_date + dt.timedelta(days=diff_day_1)
                weekday = np.random.choice(weekdays)
                imposters = users.copy()
                imposters = imposters[~(imposters == user)]
                imposter = np.random.choice(imposters)

                t_Q = pd.DataFrame(
                    data.loc[
                        (data["User"] == user)
                        & (data["state"] == "ON")
                        & (data["date"] > first_date[0] + dt.timedelta(seconds=1))
                        & (data["date"] < last_date[0])
                        & (data["weekday"] == weekday)
                    ]["battery"].copy()
                )
                q1 = t_Q.battery.quantile(0.25)
                q3 = t_Q.battery.quantile(0.75)
                t_Q.query("@q1 < battery < @q3")
                # t_Q = np.array(t_Q)
                # t_Q = t_Q.reshape(len(t_Q), 1)
                # try:
                #     pred = clf.fit_predict(t_Q)
                # except:
                #     continue
                # t_Q = t_Q[np.where(pred > 0)]
                # t_Q = pd.DataFrame(t_Q, columns=["battery"])

                # freq_1 = t_Q["battery"].value_counts(bins=bins, sort=False)
                # t_Q = freq_1 / t_Q["battery"].count()

                t_T = pd.DataFrame(
                    data.loc[
                        (data["User"] == user)
                        & (data["state"] == "ON")
                        & (
                            data["date"]
                            > (first_date[0] - dt.timedelta(days=diff_day_2))
                        )
                        & (data["date"] < (first_date[0]))
                        & (data["weekday"] == weekday)
                    ]["battery"].copy()
                )
                q1 = t_T.battery.quantile(0.25)
                q3 = t_T.battery.quantile(0.75)
                t_T.query("@q1 < battery < @q3")
                # t_T = np.array(t_T)
                # t_T = t_T.reshape(len(t_T), 1)
                # try:
                #     pred = clf.fit_predict(t_T)
                # except:
                #     continue
                # t_T = t_T[np.where(pred > 0)]
                # t_T = pd.DataFrame(t_T, columns=["battery"])

                #                 freq_2 = t_T["battery"].value_counts(bins=bins, sort=False)
                #                 t_T = freq_2 / t_T["battery"].count()

                i_Q = pd.DataFrame(
                    data.loc[
                        (data["User"] == imposter)
                        & (data["state"] == "ON")
                        & (data["date"] > first_date[0] + dt.timedelta(seconds=1))
                        & (data["date"] < last_date[0])
                        & (data["weekday"] == weekday)
                    ]["battery"].copy()
                )
                q1 = i_Q.battery.quantile(0.25)
                q3 = i_Q.battery.quantile(0.75)
                i_Q.query("@q1 < battery < @q3")
                # i_Q = np.array(i_Q)
                # i_Q = i_Q.reshape(len(i_Q), 1)
                # try:
                #     pred = clf.fit_predict(i_Q)
                # except:
                #     continue
                # i_Q = i_Q[np.where(pred > 0)]
                # i_Q = pd.DataFrame(i_Q, columns=["battery"])
                #                 freq_3 = i_Q["battery"].value_counts(bins=bins, sort=False)
                #                 i_Q = freq_3 / i_Q["battery"].count()
                try:
                    hoge, foo, ddtw_1 = DDTW(t_T, t_Q)
                    score_G.append(ddtw_1)
                    hoge, foo, ddtw_2 = DDTW(t_T, i_Q)
                    score_I.append(ddtw_2)
                except:
                    pass

        grand_truth = np.concatenate((np.ones(len(score_G)), np.zeros(len(score_I))))
        score = np.concatenate((score_G, score_I))
        far, tpr, threshold = metrics.roc_curve(grand_truth, score)
        auc = metrics.auc(far, tpr)
        frr = 1.0 - tpr
        eer = far[np.where((far - frr) < 0)[0][-1]]
        print(eer)

0.5126353790613718
0.5157894736842106
0.5183129855715871
0.5103766333589547
0.5140341221794166
0.5192953020134228
0.5146750524109015
0.5168860904407556
0.5204032645223235


In [11]:
first_date_cand = pd.date_range("2022-01-01", "2022-02-01")
score_G = []
score_I = []

days = [14, 21, 30]
# weekdays = df["weekday"].unique()
data = df
clf = LocalOutlierFactor(n_neighbors=2)
bins = np.linspace(0, 100, 11)

for a in range(len(days)):
    diff_day_1 = days[a]
    for b in range(len(days)):
        diff_day_2 = days[b]
        for h in range(len(users)):
            user = users[h]
            for i in range(0, 101):
                first_date = first_date_cand[
                    np.random.randint(0, len(first_date_cand), 1)
                ]
                last_date = first_date + dt.timedelta(days=diff_day_1)
                # weekday = np.random.choice(weekdays)
                imposters = users.copy()
                imposters = imposters[~(imposters == user)]
                imposter = np.random.choice(imposters)

                t_Q = pd.DataFrame(
                    data.loc[
                        (data["User"] == user)
                        & (data["state"] == "ON")
                        & (data["date"] > first_date[0] + dt.timedelta(seconds=1))
                        & (data["date"] < last_date[0])
                        # & (data["weekday"] == weekday)
                    ]["battery"].copy()
                )
                q1 = t_Q.battery.quantile(0.25)
                q3 = t_Q.battery.quantile(0.75)
                t_Q.query("@q1 < battery < @q3")
                # t_Q = np.array(t_Q)
                # t_Q = t_Q.reshape(len(t_Q), 1)
                # try:
                #     pred = clf.fit_predict(t_Q)
                # except:
                #     continue
                # t_Q = t_Q[np.where(pred > 0)]
                # t_Q = pd.DataFrame(t_Q, columns=["battery"])

                # freq_1 = t_Q["battery"].value_counts(bins=bins, sort=False)
                # t_Q = freq_1 / t_Q["battery"].count()

                t_T = pd.DataFrame(
                    data.loc[
                        (data["User"] == user)
                        & (data["state"] == "ON")
                        & (
                            data["date"]
                            > (first_date[0] - dt.timedelta(days=diff_day_2))
                        )
                        & (data["date"] < (first_date[0]))
                        # & (data["weekday"] == weekday)
                    ]["battery"].copy()
                )
                q1 = t_T.battery.quantile(0.25)
                q3 = t_T.battery.quantile(0.75)
                t_T.query("@q1 < battery < @q3")
                # t_T = np.array(t_T)
                # t_T = t_T.reshape(len(t_T), 1)
                # try:
                #     pred = clf.fit_predict(t_T)
                # except:
                #     continue
                # t_T = t_T[np.where(pred > 0)]
                # t_T = pd.DataFrame(t_T, columns=["battery"])

                #                 freq_2 = t_T["battery"].value_counts(bins=bins, sort=False)
                #                 t_T = freq_2 / t_T["battery"].count()

                i_Q = pd.DataFrame(
                    data.loc[
                        (data["User"] == imposter)
                        & (data["state"] == "ON")
                        & (data["date"] > first_date[0] + dt.timedelta(seconds=1))
                        & (data["date"] < last_date[0])
                        # & (data["weekday"] == weekday)
                    ]["battery"].copy()
                )
                q1 = i_Q.battery.quantile(0.25)
                q3 = i_Q.battery.quantile(0.75)
                i_Q.query("@q1 < battery < @q3")
                # i_Q = np.array(i_Q)
                # i_Q = i_Q.reshape(len(i_Q), 1)
                # try:
                #     pred = clf.fit_predict(i_Q)
                # except:
                #     continue
                # i_Q = i_Q[np.where(pred > 0)]
                # i_Q = pd.DataFrame(i_Q, columns=["battery"])
                #                 freq_3 = i_Q["battery"].value_counts(bins=bins, sort=False)
                #                 i_Q = freq_3 / i_Q["battery"].count()
                try:
                    hoge, foo, ddtw_1 = DDTW(t_T, t_Q)
                    score_G.append(ddtw_1)
                    hoge, foo, ddtw_2 = DDTW(t_T, i_Q)
                    score_I.append(ddtw_2)
                except:
                    pass

        grand_truth = np.concatenate((np.ones(len(score_G)), np.zeros(len(score_I))))
        score = np.concatenate((score_G, score_I))
        far, tpr, threshold = metrics.roc_curve(grand_truth, score)
        auc = metrics.auc(far, tpr)
        frr = 1.0 - tpr
        eer = far[np.where((far - frr) < 0)[0][-1]]
        print(eer)

0.5693069306930693
0.5699257425742574
0.554042904290429
0.5544554455445545
0.558910891089109
0.5577557755775577
0.551980198019802
0.5512066831683168
0.5506050605060506
